In [ ]:
import os, sys
sys.path.append('../') 
sys.path.append('../../') 
import toml
import pandas as pd
import os, json
import time
import configure as cf
from helper import evaluate, utils

### 1. Convert the corpora to tsv format

In [ ]:
project_dir = "path_to_your_project"
corpus_path = [
f"{project_dir}/data/dataset_files/spotify_120_60_time_segment.jsonl",
f"{project_dir}/data/dataset_files/whisperX-base_120_60_time_segment.jsonl",
f"{project_dir}/data/dataset_files/whisperX-large-v3_120_60_time_segment.jsonl",
f"{project_dir}/data/dataset_files/silero-small_120_60_time_segment.jsonl",
f"{project_dir}/data/dataset_files/silero-large_120_60_time_segment.jsonl",
]

splade_corpus_dir = f"{project_dir}/data/splade/corpus"

for c_path in corpus_path:
    filename = os.path.basename(c_path).split(".")[0] + ".tsv" # save file in tsv format
    save_path = f"{splade_corpus_dir}/{filename}"
    
    print(f"reading corpus from {c_path}")
    df = utils.read_jsonl(c_path)
    df[["id", "seg_words"]].to_csv(save_path, index=False, header=False, sep='\t')
    print(f"File was converted and saved successfully to {save_path}")

tsv_paths = [
f"{project_dir}/data/splade_corpus/spotify_120_60_time_segment.tsv",
f"{project_dir}/data/splade_corpus/whisperX-base_120_60_time_segment.tsv",
f"{project_dir}/data/splade_corpus/whisperX-large-v3_120_60_time_segment.tsv",
f"{project_dir}/data/splade_corpus/silero-small_120_60_time_segment.tsv",
f"{project_dir}/data/splade_corpus/silero-large_120_60_time_segment.tsv",
]

### 2. Prepare the config file. 

For the spotify corpus, this is its content (saved in the name "spotify_splade++_cocondenser_ensembledistil_monogpu.yaml"):


/# FILES
defaults: # (these specify which config FILES to use)
  ############## TRAIN ###################################
  - train/config: splade_monogpu
  - train/data: distil_from_ensemble
  - train/model: splade_cocondenser
  ############## INDEX ###################################
  - index: msmarco
  ############## RETRIEVE ################################
  - retrieve_evaluate: all
  ############### FLOPS ##################################
  - flops: msmarco


/# Direct PARAMETER setting
config:
  loss: DistilMarginMSE
  regularizer:
    FLOPS:
      lambda_q: 0.5
      lambda_d: 0.4
      T: 50000
      targeted_rep: rep
      reg: FLOPS
  checkpoint_dir: models/cocondenser_ensemble_distil_monogpu/checkpoint
  index_dir: models/cocondenser_ensemble_distil_monogpu/index
  out_dir: models/cocondenser_ensemble_distil_monogpu/out
  max_length: 510


data:
  COLLECTION_PATH: ${project_dir}/data/splade_corpus/spotify_120_60_time_segment.tsv



### 3. Run indexing 

tr_model="spotify"
splade_model="splade-cocondenser-ensembledistil"
data_dir="${project_dir}/data"
index_dir="${data_dir}/indexes/${tr_model}/${splade_model}"
run_dir="${data_dir}/runs/${tr_model}/${splade_model}"
corpus="${data_dir}/splade_corpus/${tr_model}_120_60_time_segment.tsv"

export CUDA_VISIBLE_DEVICES=0
export PYTHONPATH=$PYTHONPATH:$(pwd)
export SPLADE_CONFIG_NAME="spotify_spladepp_cocondenser_ensembledistil_monogpu.yaml"

python3 -m splade.index \
init_dict.model_type_or_dir=naver/${splade_model} \
config.pretrained_no_yamlconfig=true \
config.index_dir=${index_dir} \
data.COLLECTION_PATH=$corpus

python -m splade.retrieve \
init_dict.model_type_or_dir=naver/${splade_model}  \
config.pretrained_no_yamlconfig=true \
config.index_dir=${index_dir} \
config.out_dir=${run_dir} \
index=msmarco \  # <--- added
retrieve_evaluate=msmarco # <--- added